In [1]:
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from konlpy.tag import Okt
import sys
import os
import pycurl
import re
from io import BytesIO
import json
np.random.seed(1)
tf.set_random_seed(1)

In [2]:
okt=Okt()

In [3]:
word2vec = Word2Vec.load("./../model/word2vec/word2vec.model")

C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
def embedding(word):
    if word in word2vec.wv.vocab:
        return word2vec.wv[word]
    else:
        return np.random.normal(size=(300))

In [5]:
max_length = 100
input_size = 300
batch_size = 1

In [7]:
c = pycurl.Curl()
c.setopt(c.URL, "http://101.101.164.175:8501/v1/models/BiLSTM:predict")
c.setopt(pycurl.HTTPHEADER, ['Accept: application/json'])
while True:
    X = input("리뷰를 입력하세요.(exit => 종료)")
    if X == "exit":
        break
    X = [X]
    token_X = [["/".join(tag) for tag in okt.pos(sentence, norm=True, stem=True)] for sentence in X]
    batch_X = [[embedding(word) for word in sentence] for sentence in token_X]
    batch_X_padded = np.zeros(shape=(batch_size, max_length, input_size))
    for b in range(batch_size):
        batch_X_padded[b, :len(batch_X[b])] = batch_X[b]
    seq_len_ = [len(x) for x in X]

    data = json.dumps({
        "inputs": {
            "input_X": batch_X_padded.tolist(),
            "dropout_keep_prob": 1.0,
            "seq_len": seq_len_
        }
    })

#         buffer 초기화가 잘 안됨
    buffer = BytesIO()
    
    c.setopt(c.POST, True)
    c.setopt(c.POSTFIELDS, data)
    c.setopt(c.WRITEFUNCTION, buffer.write)

    c.perform()
#         print(c.getinfo(pycurl.RESPONSE_CODE))

    body = buffer.getvalue()
    pred = json.loads(body.decode('utf8'))["outputs"][0]

    print("리뷰 \"{}\" 은 약 {}%의 확률로 긍정 리뷰입니다.".format(X[0], int(pred[0]*100)))

리뷰를 입력하세요.(exit => 종료)재미없다
리뷰 "재미없다" 은 약 2%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)와. 진짜 재미있다.
리뷰 "와. 진짜 재미있다." 은 약 94%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)너무 재밌어요. 대박이네.
리뷰 "너무 재밌어요. 대박이네." 은 약 98%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)exit


In [100]:
# 랜덤값 입력 테스트
import pycurl
import re
from io import StringIO
from io import BytesIO
import json

buffer = BytesIO()
c = pycurl.Curl()
c.setopt(c.URL, "http://localhost:8501/v1/models/BiLSTM:predict")
c.setopt(pycurl.HTTPHEADER, ['Accept: application/json'])

data = json.dumps({
    "inputs": {
        "input_X": np.zeros(shape=[1, max_length, input_size]).tolist(),
        "dropout_keep_prob": 1.0,
        "seq_len": [1]
    }
})

c.setopt(c.POST, True)
c.setopt(c.POSTFIELDS, data)
c.setopt(c.WRITEFUNCTION, buffer.write)

c.perform()
print(c.getinfo(pycurl.RESPONSE_CODE))

body = buffer.getvalue()
pred = json.loads(body.decode('utf8'))["outputs"][0][0]
print(pred)
buffer.truncate(0)

200
0.478981048


0